# Modeling

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers import Dense, Dropout, Normalization, BatchNormalization, LayerNormalization, Input
from tcn import TCN, tcn_full_summary
from catboost import CatBoostRegressor


from sklearn.preprocessing import MinMaxScaler

## Prepare the Data Sets
We want to create two separate datasets; one for our CatBoost model and one for our Keras models.

In [28]:
df = pd.read_parquet("../data/structured/general/combined_data.parquet")
df = df.dropna()
df

,Unit_4_Power,Unit_4_Reactive Power,Turbine_Guide Vane Opening,Turbine_Pressure Drafttube,Turbine_Pressure Spiral Casing,Turbine_Rotational Speed,Bolt_1_Tensile,Bolt_2_Tensile,Bolt_3_Tensile,Bolt_4_Tensile,...,Power / vane opening,seconds_since_last_data,seconds_since_last_start,Power / Drafttube pressure,Bolt_1_Tensile_adj,Bolt_2_Tensile_adj,Bolt_3_Tensile_adj,Bolt_4_Tensile_adj,Bolt_5_Tensile_adj,Bolt_6_Tensile_adj
timepoints,,,,,,,,,,,,,,,,,,,,,
1970-12-19 09:51:45,262.104319,3.344630,82.277248,173.989815,5311.219755,107.964273,1598.477449,1480.989528,1684.261611,1601.366508,...,3.185623,1.0,1.0,1.506435,115.477449,43.989528,72.261611,3.366508,6.588478,38.823883
1970-12-19 09:51:46,262.004330,3.790223,82.274520,174.024413,5311.640329,107.964269,1598.479316,1481.003188,1684.270504,1601.374254,...,3.184514,1.0,2.0,1.505561,115.479316,44.003188,72.270504,3.374254,6.583464,38.841318
1970-12-19 09:51:47,261.904340,4.235817,82.271792,174.059012,5312.060902,107.964264,1598.490184,1481.028827,1684.270683,1601.383179,...,3.183404,1.0,3.0,1.504687,115.490184,44.028827,72.270683,3.383179,6.581384,38.843245
1970-12-19 09:51:48,261.804351,4.064759,82.269064,174.153819,5312.405938,107.964259,1598.494073,1481.059017,1684.271062,1601.378391,...,3.182294,1.0,4.0,1.503294,115.494073,44.059017,72.271062,3.378391,6.591746,38.872300
1970-12-19 09:51:49,261.704362,3.170510,82.266336,174.422046,5312.533396,107.964254,1598.498916,1481.075521,1684.276622,1601.380601,...,3.181184,1.0,5.0,1.500409,115.498916,44.075521,72.276622,3.380601,6.607884,38.924469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1971-01-25 11:06:44,308.716025,3.974309,94.425666,157.927905,5280.929965,108.057498,1637.386115,1504.557822,1701.651420,1606.276545,...,3.269408,1.0,19788.0,1.954791,154.386115,67.557822,89.651420,8.276545,11.704071,54.014705
1971-01-25 11:06:45,308.746393,4.103262,94.429003,157.974925,5280.633358,108.057492,1637.365865,1504.546091,1701.654301,1606.271877,...,3.269614,1.0,19789.0,1.954401,154.365865,67.546091,89.654301,8.271877,11.711250,54.017029
1971-01-25 11:06:46,308.776762,4.472929,94.432340,158.021945,5280.336751,108.057486,1637.384133,1504.538696,1701.656143,1606.250028,...,3.269820,1.0,19790.0,1.954012,154.384133,67.538696,89.656143,8.250028,11.699142,54.002008


In [29]:
y_cols = [c for c in df if c.endswith("Tensile")]
adj_cols = [c for c in df if c.endswith("Tensile_adj")]

### Create CatBoost dataset

In [30]:
extra_cols = ["seconds_since_start", "month", "day_of_month", "day_of_week"]

In [31]:
X_cols = [
    "days_since_start",
    "Turbine_Pressure Drafttube",
    "seconds_since_last_start",
    "Turbine_Pressure Spiral Casing",
    "Netto Power",
    "Turbine_Rotational Speed",
    "Unit_4_Power"
]

In [32]:
lookback = 0

cX = df[X_cols]
#X_cols = cX.drop(columns=extra_cols).columns
for i in range(1, lookback+1):
    cX.loc[:, [f"{c} (t-{i})" for c in X_cols]] = cX[X_cols].shift(i).rename(columns={c: f"{c} (t-{i})" for c in X_cols})

cy = df[y_cols]

cX

,days_since_start,Turbine_Pressure Drafttube,seconds_since_last_start,Turbine_Pressure Spiral Casing,Netto Power,Turbine_Rotational Speed,Unit_4_Power
timepoints,,,,,,,
1970-12-19 09:51:45,0.0,173.989815,1.0,5311.219755,258.759689,107.964273,262.104319
1970-12-19 09:51:46,0.0,174.024413,2.0,5311.640329,258.214106,107.964269,262.004330
1970-12-19 09:51:47,0.0,174.059012,3.0,5312.060902,257.668524,107.964264,261.904340
1970-12-19 09:51:48,0.0,174.153819,4.0,5312.405938,257.739592,107.964259,261.804351
1970-12-19 09:51:49,0.0,174.422046,5.0,5312.533396,258.533851,107.964254,261.704362
...,...,...,...,...,...,...,...
1971-01-25 11:06:44,5.0,157.927905,19788.0,5280.929965,304.741716,108.057498,308.716025
1971-01-25 11:06:45,5.0,157.974925,19789.0,5280.633358,304.643131,108.057492,308.746393
1971-01-25 11:06:46,5.0,158.021945,19790.0,5280.336751,304.303833,108.057486,308.776762


In [33]:
def train_test_split(X, y, test_percent=0.1, offset_percent=0):
    
    test_start = int(len(df) * offset_percent)
    test_end = int(len(df) * (offset_percent + test_percent))

    X_train, X_test = X.iloc[:test_start], X.iloc[test_start:test_end]
    y_train, y_test = y.iloc[:test_start], y.iloc[test_start:test_end]
    
    return X_train, X_test, y_train, y_test

cX_train, cX_test, cy_train, cy_test = train_test_split(cX, cy, test_percent=0.1, offset_percent=0.9)

### CatBoost Modeling

In [34]:
def train_catboost(X_train, y_train, eval_set=None, params={}):
    
    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train, eval_set=eval_set)
    
    return model

In [35]:
def plot_error(X_test, y_test, model):

    y_test = y_tests[j].copy()

    pred = model.predict(X_test)

    plt.subplots(figsize=(15, 10))
    plt.scatter(y_test.index, y_test, label="real", s=2)
    plt.scatter(y_test.index, pred, label="pred", s=2)
    plt.legend()
    plt.show()

In [38]:
def cv_catboost(X, y, n=4, start_offset=0.5, verbose=False, params={}):

    test_percent = (1 - start_offset) / n

    all_results = []
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X,
                                                            y,
                                                            test_percent = test_percent,
                                                            offset_percent = start_offset + i*test_percent)
        y_trains = [y_train[c] for c in y_train]
        y_tests = [y_test[c] for c in y_test]

        results = []
        for j in range(len(y_trains)):
            model = CatBoostRegressor(**params)
            model.fit(X_train, y_trains[j], eval_set=(X_test, y_tests[j]), verbose=verbose)

            pred = model.predict(X_test)
            mape = 100 * ((y_tests[j] - pred).abs() / y_tests[j]).mean()
            results.append(mape)
            print(f"iteration {i}, bolt {j}: MAPE={mape}")
        all_results.append(results)
    all_results = np.array(all_results)
    
    return all_results

params = {
    "loss_function": "MAPE",
    "iterations": 500,
    "depth": 5
}

#results = cv_catboost(cX, cy, params=params, verbose=True)
#print()

# Feature optimazion

In [41]:
def feature_opt(test_cols, best_result, result_cols):
    start_result = best_result
    if len(test_cols) > 1:
        for c in test_cols:
            print(f'Dropping {c} from {X_cols}')
            result = cv_catboost(cX[test_cols].drop(columns=[c]), cy, params=params, verbose=True)
            if result.mean() < best_result:
                best_result = result.mean()
                result_cols = X_cols.remove(c)
        if best_result == start_result:
            return best_result, result_cols
        print(f'Best mean result (So far):\nfor {best_result} in with columns: {result_cols}')
        return feature_opt(result_cols, best_result, result_cols)
    else:
        return best_result, result_cols

best_result, result_cols = feature_opt(X_cols, np.inf, [])

print(f'Best mean result for {best_result}\nFor columns: result_cols')
    


Dropping days_since_start from ['days_since_start', 'Turbine_Pressure Drafttube', 'seconds_since_last_start', 'Turbine_Pressure Spiral Casing', 'Netto Power', 'Turbine_Rotational Speed', 'Unit_4_Power']


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.0013959	test: 0.0063018	best: 0.0063018 (0)	total: 136ms	remaining: 1m 7s
1:	learn: 0.0013688	test: 0.0062682	best: 0.0062682 (1)	total: 216ms	remaining: 53.7s
2:	learn: 0.0013425	test: 0.0062361	best: 0.0062361 (2)	total: 295ms	remaining: 48.9s
3:	learn: 0.0013204	test: 0.0062242	best: 0.0062242 (3)	total: 372ms	remaining: 46.1s
4:	learn: 0.0012986	test: 0.0062123	best: 0.0062123 (4)	total: 455ms	remaining: 45s
5:	learn: 0.0012776	test: 0.0062012	best: 0.0062012 (5)	total: 533ms	remaining: 43.9s
6:	learn: 0.0012560	test: 0.0061699	best: 0.0061699 (6)	total: 619ms	remaining: 43.6s
7:	learn: 0.0012340	test: 0.0061391	best: 0.0061391 (7)	total: 705ms	remaining: 43.4s
8:	learn: 0.0012127	test: 0.0061094	best: 0.0061094 (8)	total: 784ms	remaining: 42.8s
9:	learn: 0.0011918	test: 0.0060804	best: 0.0060804 (9)	total: 865ms	remaining: 42.4s
10:	learn: 0.0011719	test: 0.0060523	best: 0.0060523 (10)	total: 953ms	remaining: 42.4s
11:	learn: 0.0011515	test: 0.0060261	best: 0.0060261 (